#### Day 6: The HAVING Clause 🎉

Today we explore the HAVING clause, which is used to filter results after they have been grouped.

#### 1. Database Connection

In [1]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="30_Days_SQL"
)
cursor = conn.cursor()
print("Connected to '30_Days_SQL'!")

Connected to 'sql_30days_challenge'!


---

#### Practice Exercises 📖

#### 1. Customers whose total order value exceeds $5000
(We'll add some high-value orders to test)

In [2]:
extra_orders = [
    (6, 104, '2023-11-01', 3000.00),
    (7, 104, '2023-11-02', 2500.00),
    (8, 105, '2023-11-03', 1000.00)
]
cursor.executemany("INSERT IGNORE INTO orders VALUES (%s, %s, %s, %s)", extra_orders)
conn.commit()

query = """
SELECT customer_id, SUM(order_total) AS total_value 
FROM orders 
GROUP BY customer_id 
HAVING total_value > 5000
"""
pd.read_sql(query, conn)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_15628\329714176.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,customer_id,total_value
0,104,5500.0


#### 2. Products where the average price is greater than $50

In [3]:
query = """
SELECT category, AVG(price) AS average_price 
FROM products 
GROUP BY category 
HAVING average_price > 50
"""
pd.read_sql(query, conn)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_15628\1539709902.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,category,average_price
0,Electronics,700.0


#### 3. Authors with more than 3 published books and an average rating above 4.5
(First, create the library table)

In [4]:
cursor.execute("DROP TABLE IF EXISTS library")
cursor.execute('''
CREATE TABLE library (
    book_id INT PRIMARY KEY AUTO_INCREMENT,
    author VARCHAR(100),
    book_title VARCHAR(100),
    rating DECIMAL(3, 2)
)
''')

library_data = [
    ('Author A', 'Book 1', 4.8),
    ('Author A', 'Book 2', 4.7),
    ('Author A', 'Book 3', 4.9),
    ('Author A', 'Book 4', 4.6),
    ('Author B', 'Book 5', 4.2),
    ('Author B', 'Book 6', 4.3),
    ('Author B', 'Book 7', 4.1),
    ('Author B', 'Book 8', 4.4)
]
cursor.executemany("INSERT INTO library (author, book_title, rating) VALUES (%s, %s, %s)", library_data)
conn.commit()

query = """
SELECT author, COUNT(*) AS book_count, AVG(rating) AS average_rating 
FROM library 
GROUP BY author 
HAVING book_count > 3 AND average_rating > 4.5
"""
pd.read_sql(query, conn)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_15628\2781830773.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,author,book_count,average_rating
0,Author A,4,4.75


#### 4. Departments where the total salary exceeds $100,000

In [5]:
query = """
SELECT department, SUM(salary) AS total_salary 
FROM employees 
GROUP BY department 
HAVING total_salary > 100000
"""
pd.read_sql(query, conn)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_15628\3442973729.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,department,total_salary
0,IT,130000.0


In [6]:
conn.close()